Import necessary libraries

In [17]:
import numpy as py
import pandas as pd

Read the election results from csv file

In [18]:
election_results = pd.read_csv('Results.csv')

Read the New constituencies file which is an excel file

In [20]:
file = 'New Constituencies.xlsx'
xl = pd.ExcelFile(file)
new_constituencies = xl.parse('Sheet9')

Separate the "Validated" and "Preliminary" election results into two separate datasets. For this project, we will only use the Validated election results

In [21]:
results_val = election_results.loc[election_results['Type of results'] == 'validated']

results_prelim = election_results.loc[election_results['Type of results'] == 'preliminary']


Group the election results by the political party and calculate the votes obtained per political party

In [110]:
pei=results_val.groupby('Political affiliation')['Votes obtained'].sum()
pei=pei.to_frame()

STEP 1: Calculate the following:
Maximum votes 
STEP 2: The party with maximum votes is then assigned a value of 1 to the number of seats it earns

In [121]:
first_max = pei['Votes obtained'].max()
seat = 0
pei['Seats_Seat1'] = seat
pei.reset_index(inplace=True)
pei.loc[pei['Votes obtained'] == first_max, 'Seats_Seat1'] = 1

STEP 3: Using D'Hondt formula, the number of votes for the second seat for each party is calculated as:
Number of votes(seat1)/(1+Number of Seats received in Seat 1)
STEP 4: Further, to assign the number of seats a party earns, the following formula is used:
if a party is identified as having max number of votes(as per the calculation in STEP 3) then:
number of seats a party receives is equal to number of seats(from STEP 2) + 1

In [ ]:
pei['Votes_Seat2']=pei['Votes obtained']/(1+pei['Seats_Seat1'])
second_max = pei['Votes_Seat2'].max()
seat2 = 0
pei['Seats_Seat2'] = pei['Seats_Seat1']
pei.loc[pei['Votes_Seat2'] == second_max, 'Seats_Seat2'] = pei['Seats_Seat1'] + 1

STEP 5: Using D'Hondt formula, the number of votes for the third seat for each party is calculated as:
Number of votes(seat1)/(1+Number of Seats received in Seat 2)
STEP 6: Further, to assign the number of seats a party earns, the following formula is used:
if a party is identified as having max number of votes(as per the calculation in STEP 5) then:
number of seats a party receives is equal to number of seats(from STEP 4) + 1

In [123]:
pei['Votes_Seat3']=pei['Votes_Seat2']/(1+pei['Seats_Seat2'])
third_max = pei['Votes_Seat3'].max()
seat3 = 0
pei['Seats_Seat3'] = pei['Seats_Seat2']
pei.loc[pei['Votes_Seat3'] == third_max, 'Seats_Seat3'] = pei['Seats_Seat2'] + 1

STEP 7: Using D'Hondt formula, the number of votes for the fourth seat for each party is calculated as:
Number of votes(seat1)/(1+Number of Seats received in Seat 4)
STEP 8: Further, to assign the number of seats a party earns, the following formula is used:
if a party is identified as having max number of votes(as per the calculation in STEP 7) then:
number of seats a party receives is equal to number of seats(from STEP 6) + 1

In [125]:
pei['Votes_Seat4']=pei['Votes_Seat3']/(1+pei['Seats_Seat3'])
fourth_max = pei['Votes_Seat4'].max()
seat4 = 0
pei['Seats_Seat4'] = pei['Seats_Seat3']
pei.loc[pei['Votes_Seat4'] == fourth_max, 'Seats_Seat4'] = pei['Seats_Seat3'] + 1

In [127]:
pei.to_csv('Summary.csv')

In [22]:
new_constituencies.head()

,Provinces,Old Constituencies,New Constituencies
0,Cardigan,Prince Edward Island,Prince Edward Island
1,Charlottetown,Prince Edward Island,Prince Edward Island
2,Egmont,Prince Edward Island,Prince Edward Island
3,Malpeque,Prince Edward Island,Prince Edward Island
4,Avalon,Newfoundland and Labrador,Newfoundland and Labrador


In [23]:
results_val.head()

,Electoral district number,Electoral district name,Type of results,Surname,Middle name(s),Given name,Political affiliation,Appartenance politique,Votes obtained,% Votes obtained,Rejected ballots,Total number of ballots cast
4,10001,Avalon,validated,Chapman,NaN,Matthew,Conservative,Conservateur,12855.0,31.1,397.0,41731.0
5,10001,Avalon,validated,Malone,NaN,Greg,Green Party,Parti Vert,2215.0,5.4,397.0,41731.0
6,10001,Avalon,validated,McDonald,NaN,Kenneth,Liberal,Libéral,19122.0,46.3,397.0,41731.0
7,10001,Avalon,validated,Movelle,NaN,Lea Mary,NDP-New Democratic Party,NPD-Nouveau Parti démocratique,7142.0,17.3,397.0,41731.0
12,10002,Bonavista--Burin--Trinity,validated,Cooper,NaN,Matthew,NDP-New Democratic Party,NPD-Nouveau Parti démocratique,3855.0,12.0,609.0,32788.0


In [24]:
df_combo = pd.merge(results_val,new_constituencies, left_on="Electoral district name",right_on ="Provinces")

In [26]:
df_combo.tail()

,Electoral district number,Electoral district name,Type of results,Surname,Middle name(s),Given name,Political affiliation,Appartenance politique,Votes obtained,% Votes obtained,Rejected ballots,Total number of ballots cast,Provinces,Old Constituencies,New Constituencies
958,61001,Northwest Territories,validated,Quinlan,NaN,Luke,People's Party,Parti populaire,296.0,1.8,125.0,16416.0,Northwest Territories,Territories,Northwest Territories
959,62001,Nunavut,validated,Aglukkaq,NaN,Leona,Conservative,Conservateur,2469.0,26.1,88.0,9542.0,Nunavut,Territories,Nunavut
960,62001,Nunavut,validated,Pizzo Lyall,NaN,Megan,Liberal,Libéral,2918.0,30.9,88.0,9542.0,Nunavut,Territories,Nunavut
961,62001,Nunavut,validated,Qaqqaq,NaN,Mumilaaq,NDP-New Democratic Party,NPD-Nouveau Parti démocratique,3861.0,40.8,88.0,9542.0,Nunavut,Territories,Nunavut
962,62001,Nunavut,validated,Roy,NaN,Douglas,Green Party,Parti Vert,206.0,2.2,88.0,9542.0,Nunavut,Territories,Nunavut


In [29]:
df_combo=df_combo.drop(['Provinces','Appartenance politique','% Votes obtained'], axis = 1)

In [30]:
df_combo.head()

,Electoral district number,Electoral district name,Type of results,Surname,Middle name(s),Given name,Political affiliation,Votes obtained,Rejected ballots,Total number of ballots cast,Old Constituencies,New Constituencies
0,10001,Avalon,validated,Chapman,NaN,Matthew,Conservative,12855.0,397.0,41731.0,Newfoundland and Labrador,Newfoundland and Labrador
1,10001,Avalon,validated,Malone,NaN,Greg,Green Party,2215.0,397.0,41731.0,Newfoundland and Labrador,Newfoundland and Labrador
2,10001,Avalon,validated,McDonald,NaN,Kenneth,Liberal,19122.0,397.0,41731.0,Newfoundland and Labrador,Newfoundland and Labrador
3,10001,Avalon,validated,Movelle,NaN,Lea Mary,NDP-New Democratic Party,7142.0,397.0,41731.0,Newfoundland and Labrador,Newfoundland and Labrador
4,10004,Labrador,validated,Colbourne,NaN,Tyler,Green Party,224.0,139.0,11558.0,Newfoundland and Labrador,Newfoundland and Labrador


In [32]:
df_combo1 = df_combo.drop(['Type of results','Surname','Given name','Rejected ballots','Total number of ballots cast'], axis = 1)

In [33]:
df_combo1.head()

,Electoral district number,Electoral district name,Middle name(s),Political affiliation,Votes obtained,Old Constituencies,New Constituencies
0,10001,Avalon,NaN,Conservative,12855.0,Newfoundland and Labrador,Newfoundland and Labrador
1,10001,Avalon,NaN,Green Party,2215.0,Newfoundland and Labrador,Newfoundland and Labrador
2,10001,Avalon,NaN,Liberal,19122.0,Newfoundland and Labrador,Newfoundland and Labrador
3,10001,Avalon,NaN,NDP-New Democratic Party,7142.0,Newfoundland and Labrador,Newfoundland and Labrador
4,10004,Labrador,NaN,Green Party,224.0,Newfoundland and Labrador,Newfoundland and Labrador


In [34]:
for col in df_combo1.columns: 
    print(col)

Electoral district number
Electoral district name
Middle name(s) 
Political affiliation
Votes obtained
Old Constituencies
New Constituencies


In [35]:
df_combo1 = df_combo1.drop(["Middle name(s) "], axis = 1)

In [36]:
df_combo1.head()

,Electoral district number,Electoral district name,Political affiliation,Votes obtained,Old Constituencies,New Constituencies
0,10001,Avalon,Conservative,12855.0,Newfoundland and Labrador,Newfoundland and Labrador
1,10001,Avalon,Green Party,2215.0,Newfoundland and Labrador,Newfoundland and Labrador
2,10001,Avalon,Liberal,19122.0,Newfoundland and Labrador,Newfoundland and Labrador
3,10001,Avalon,NDP-New Democratic Party,7142.0,Newfoundland and Labrador,Newfoundland and Labrador
4,10004,Labrador,Green Party,224.0,Newfoundland and Labrador,Newfoundland and Labrador


In [39]:
df_combo1.groupby(['Political affiliation', 'New Constituencies'])["Votes obtained"].apply(lambda x : x.astype(int).sum())

Political affiliation    New Constituencies                        
Animal Protection Party  Calgary                                       247
                         Eastern Townships                             248
                         Fredericton and North New Brunswick           286
                         Greater Sudbury and Nipissing                 282
                         Halifax and West Nova (new constituencies)    222
                                                                      ... 
VCP                      North Alberta                                 255
                         Saskatoon and North Saskatchewan              170
                         South New Brunswick                           200
                         Vancouver Island-Powell River                  46
                         Western Quebec                                194
Name: Votes obtained, Length: 412, dtype: int64

In [40]:
votesgrouped = df_combo1.groupby(['Political affiliation', 'New Constituencies'])["Votes obtained"].apply(lambda x : x.astype(int).sum())

In [43]:
votesgrouped.to_csv('Votes.csv')

C:\Users\primu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [45]:
votesgrouped=votesgrouped.to_frame()

In [46]:
type(votesgrouped)

pandas.core.frame.DataFrame

In [48]:
votesgrouped

Votes obtained
Political affiliation   New Constituencies                                        
Animal Protection Party Calgary                                                247
                        Eastern Townships                                      248
                        Fredericton and North New Brunswick                    286
                        Greater Sudbury and Nipissing                          282
                        Halifax and West Nova (new constituencies)             222
...                                                                            ...
VCP                     North Alberta                                          255
                        Saskatoon and North Saskatchewan                       170
                        South New Brunswick                                    200
                        Vancouver Island-Powell River                           46
                        Western Quebec                                         194

[412 rows x 1 columns]

In [49]:
votesgrouped.to_csv('Votes.csv')

In [53]:
votesgrouped = pd.read_csv('Vote.csv')

In [55]:
votesgrouped.head()

,Political affiliation,New Constituencies,Votes obtained
0,Animal Protection Party,Calgary,247
1,Animal Protection Party,Eastern Townships,248
2,Animal Protection Party,Fredericton and North New Brunswick,286
3,Animal Protection Party,Greater Sudbury and Nipissing,282
4,Animal Protection Party,Halifax and West Nova (new constituencies),222


In [70]:
votesgrouped[votesgrouped['Political affiliation'].str.contains("Bloc")==True]

,Political affiliation,New Constituencies,Votes obtained
11,Bloc Québécois,Centre of Quebec and Lanaudières,52077
12,Bloc Québécois,Chaudière-Appalaches,8410
13,Bloc Québécois,City of Laval,49073
14,Bloc Québécois,Eastern Townships,63547
15,Bloc Québécois,Eastern and Northern Quebec,63184
16,Bloc Québécois,Longueuil and Vicinity,25366
17,Bloc Québécois,North East Montréal Island,82810
18,Bloc Québécois,North Laurentides,64871
19,Bloc Québécois,Off-Island and South Laurentides,144154
20,Bloc Québécois,Quebec City and Vicinity,32049


In [74]:
array = ['Liberal','Conservative','Bloc Québécois','Green Party','NDP-New Democratic Party']
votesgrp1=votesgrouped.loc[votesgrouped['Political affiliation'].isin(array)]

In [75]:
votesgrp1.to_csv('Votes_major.csv',encoding='utf-8-sig')

In [76]:
pei_new=votesgrp1.groupby('Political affiliation')['Votes obtained'].sum()
pei_new=pei_new.to_frame()

In [77]:
first_max_new = pei_new['Votes obtained'].max()
seat_new = 0
pei_new['Seats_Seat1'] = seat_new
pei_new.reset_index(inplace=True)
pei_new.loc[pei_new['Votes obtained'] == first_max_new, 'Seats_Seat1'] = 1

In [78]:
pei_new['Votes_Seat2']=pei_new['Votes obtained']/(1+pei_new['Seats_Seat1'])
second_max_new = pei_new['Votes_Seat2'].max()
seat2_new = 0
pei_new['Seats_Seat2'] = pei_new['Seats_Seat1']
pei_new.loc[pei_new['Votes_Seat2'] == second_max_new, 'Seats_Seat2'] = pei_new['Seats_Seat1'] + 1

In [79]:
pei_new['Votes_Seat3']=pei_new['Votes_Seat2']/(1+pei_new['Seats_Seat2'])
third_max_new = pei_new['Votes_Seat3'].max()
seat3_new = 0
pei_new['Seats_Seat3'] = pei_new['Seats_Seat2']
pei_new.loc[pei_new['Votes_Seat3'] == third_max_new, 'Seats_Seat3'] = pei_new['Seats_Seat2'] + 1

In [80]:
pei_new['Votes_Seat4']=pei_new['Votes_Seat3']/(1+pei_new['Seats_Seat3'])
fourth_max_new = pei_new['Votes_Seat4'].max()
seat4_new = 0
pei_new['Seats_Seat4'] = pei_new['Seats_Seat3']
pei_new.loc[pei_new['Votes_Seat4'] == fourth_max_new, 'Seats_Seat4'] = pei_new['Seats_Seat3'] + 1

In [81]:
pei_new.to_csv('New Constituencies Summary.csv',encoding='utf-8-sig')